In [ ]:
!pip install -q gradio requests beautifulsoup4 huggingface_hub transformers torch plotly pandas python-dotenv feedparser textblob wordcloud pillow numpy

# ============================================================
# STEP 2: Import Libraries
# ============================================================
import os
import json
import requests
import gradio as gr
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from huggingface_hub import InferenceClient
from typing import List, Dict, Optional
import time
import feedparser
from textblob import TextBlob
import numpy as np
from collections import Counter
import re


In [ ]:
# STEP 3: Configuration & API Setup
# ============================================================

from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

# Initialize Hugging Face Inference Client with CHAT model
client = InferenceClient(token=HF_TOKEN)

# FIXED: Use models that support chat/conversational tasks
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"  # Fast and reliable
# Alternative: "mistralai/Mistral-7B-Instruct-v0.2"


In [ ]:
# STEP 4: FIXED NEWS RETRIEVER (with URL encoding)
# ============================================================

from urllib.parse import quote_plus

class EnhancedNewsRetriever:
    """Advanced news retrieval from multiple sources"""

    def __init__(self):
        self.sources = []
        self.cache = {}

    def fetch_google_news_rss(self, query: str, num_articles: int = 10) -> List[Dict]:
        """Fetch from Google News RSS with better parsing"""
        try:
            # FIX: Properly encode the query
            encoded_query = quote_plus(query)
            url = f"https://news.google.com/rss/search?q={encoded_query}&hl=en-US&gl=US&ceid=US:en"
            feed = feedparser.parse(url)

            articles = []
            for entry in feed.entries[:num_articles]:
                # Parse publication date
                pub_date = entry.get('published', 'Unknown')

                # Extract clean description
                description = BeautifulSoup(entry.get('summary', ''), 'html.parser').get_text()

                article = {
                    'title': entry.get('title', 'N/A'),
                    'link': entry.get('link', 'N/A'),
                    'published': pub_date,
                    'source': entry.get('source', {}).get('title', 'Google News'),
                    'description': description[:300] if description else 'No description available'
                }
                articles.append(article)

            return articles
        except Exception as e:
            print(f"Error fetching Google News: {e}")
            return []

    def fetch_bing_news(self, query: str, num_articles: int = 5) -> List[Dict]:
        """Fetch from Bing News RSS"""
        try:
            # FIX: Properly encode the query
            encoded_query = quote_plus(query)
            url = f"https://www.bing.com/news/search?q={encoded_query}&format=rss"
            feed = feedparser.parse(url)

            articles = []
            for entry in feed.entries[:num_articles]:
                article = {
                    'title': entry.get('title', 'N/A'),
                    'link': entry.get('link', 'N/A'),
                    'published': entry.get('published', 'Unknown'),
                    'source': 'Bing News',
                    'description': BeautifulSoup(entry.get('description', ''), 'html.parser').get_text()[:300]
                }
                articles.append(article)

            return articles
        except Exception as e:
            print(f"Error fetching Bing News: {e}")
            return []

    def analyze_sentiment(self, text: str) -> Dict:
        """Analyze sentiment of text"""
        try:
            analysis = TextBlob(text)
            polarity = analysis.sentiment.polarity

            if polarity > 0.1:
                sentiment = "Positive"
                score = (polarity + 1) * 50  # Scale to 0-100
            elif polarity < -0.1:
                sentiment = "Negative"
                score = (polarity + 1) * 50
            else:
                sentiment = "Neutral"
                score = 50

            return {
                'sentiment': sentiment,
                'score': score,
                'polarity': polarity
            }
        except:
            return {'sentiment': 'Neutral', 'score': 50, 'polarity': 0}

    def retrieve_news(self, topic: str, num_articles: int = 10) -> List[Dict]:
        """Main retrieval with sentiment analysis"""
        all_articles = []

        # Fetch from multiple sources
        google_articles = self.fetch_google_news_rss(topic, num_articles)
        all_articles.extend(google_articles)

        bing_articles = self.fetch_bing_news(topic, num_articles // 2)
        all_articles.extend(bing_articles)

        # Deduplicate and add sentiment
        seen_titles = set()
        unique_articles = []

        for article in all_articles:
            title_key = article['title'].lower()[:50]
            if title_key not in seen_titles:
                seen_titles.add(title_key)
                # Add sentiment analysis
                sentiment_data = self.analyze_sentiment(article['title'] + ' ' + article['description'])
                article['sentiment'] = sentiment_data
                unique_articles.append(article)

        return unique_articles[:num_articles]

In [ ]:
# STEP 5: FIXED STRATEGIC ANALYST with Chat Completion
# ============================================================

class AdvancedStrategicAnalyst:
    """Enhanced strategic analysis with proper chat API"""

    def __init__(self, client):
        self.client = client
        self.model = MODEL_NAME

    def call_llm_chat(self, system_prompt: str, user_prompt: str, max_tokens: int = 800) -> str:
        """Proper chat completion API call"""
        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]

            response = self.client.chat_completion(
                messages=messages,
                model=self.model,
                max_tokens=max_tokens,
                temperature=0.7
            )

            return response.choices[0].message.content
        except Exception as e:
            print(f"LLM Error: {e}")
            return self.fallback_analysis(user_prompt)

    def fallback_analysis(self, prompt: str) -> str:
        """Fallback if LLM fails"""
        return "Analysis in progress. Using intelligent fallback system."

    def analyze_swot(self, news_data: List[Dict], context: str) -> Dict:
        """Comprehensive SWOT Analysis"""

        if not news_data:
            return self.get_default_swot()

        # Prepare news summary with sentiment
        news_summary = "\n".join([
            f"• [{article['sentiment']['sentiment']}] {article['title']}"
            for article in news_data[:8]
        ])

        system_prompt = """You are a senior strategy consultant from McKinsey.
Analyze news and provide a detailed SWOT analysis in JSON format ONLY.
Be specific, actionable, and business-focused."""

        user_prompt = f"""Context: {context}

Recent News & Sentiment:
{news_summary}

Provide SWOT analysis as JSON (no markdown, pure JSON):
{{
    "strengths": ["specific strength 1", "specific strength 2", "specific strength 3"],
    "weaknesses": ["specific weakness 1", "specific weakness 2", "specific weakness 3"],
    "opportunities": ["specific opportunity 1", "specific opportunity 2", "specific opportunity 3"],
    "threats": ["specific threat 1", "specific threat 2", "specific threat 3"]
}}"""

        try:
            response = self.call_llm_chat(system_prompt, user_prompt, 1000)

            # Extract JSON from response
            json_match = re.search(r'\{[\s\S]*\}', response)
            if json_match:
                swot_data = json.loads(json_match.group())
                return swot_data
            else:
                return self.intelligent_swot_extraction(news_data, context)

        except Exception as e:
            print(f"SWOT Error: {e}")
            return self.intelligent_swot_extraction(news_data, context)

    def intelligent_swot_extraction(self, news_data: List[Dict], context: str) -> Dict:
        """Rule-based SWOT when LLM fails"""

        positive_articles = [a for a in news_data if a['sentiment']['sentiment'] == 'Positive']
        negative_articles = [a for a in news_data if a['sentiment']['sentiment'] == 'Negative']

        strengths = [
            f"Positive market sentiment ({len(positive_articles)} favorable news items)",
            f"Growing interest in {context}",
            "Multiple information sources tracking developments"
        ]

        weaknesses = [
            f"Market volatility indicated by {len(negative_articles)} concerning reports",
            "Regulatory uncertainty in the sector",
            "Need for more comprehensive data analysis"
        ]

        opportunities = [
            f"Emerging trends in {context} creating new markets",
            "Innovation potential based on recent developments",
            "Strategic positioning possible with current market dynamics"
        ]

        threats = [
            "Competitive pressures from industry changes",
            f"Risk factors identified in {len(negative_articles)} negative reports",
            "Market disruption possibilities"
        ]

        return {
            "strengths": strengths[:4],
            "weaknesses": weaknesses[:4],
            "opportunities": opportunities[:4],
            "threats": threats[:4]
        }

    def get_default_swot(self) -> Dict:
        """Default SWOT template"""
        return {
            "strengths": ["Data collection capability", "Multi-source analysis", "Real-time monitoring"],
            "weaknesses": ["Limited historical data", "Source dependency", "Analysis depth"],
            "opportunities": ["Market expansion", "Technology adoption", "Strategic positioning"],
            "threats": ["Market volatility", "Regulatory changes", "Competition"]
        }

    def generate_insights(self, news_data: List[Dict], topic: str, swot: Dict) -> str:
        """Generate comprehensive insights"""

        if not news_data:
            return "Insufficient data for insights generation."

        # Sentiment summary
        sentiments = [a['sentiment']['sentiment'] for a in news_data]
        sentiment_counts = Counter(sentiments)
        avg_score = np.mean([a['sentiment']['score'] for a in news_data])

        news_titles = "\n".join([f"• {a['title'][:80]}" for a in news_data[:6]])

        system_prompt = """You are a strategic business analyst. Provide clear, actionable insights
with specific examples and recommendations. Focus on business implications."""

        user_prompt = f"""Topic: {topic}

Headlines:
{news_titles}

Sentiment Analysis:
- Positive: {sentiment_counts.get('Positive', 0)} articles
- Negative: {sentiment_counts.get('Negative', 0)} articles
- Neutral: {sentiment_counts.get('Neutral', 0)} articles
- Average Score: {avg_score:.1f}/100

SWOT Summary:
- Key Strength: {swot['strengths'][0] if swot['strengths'] else 'N/A'}
- Top Opportunity: {swot['opportunities'][0] if swot['opportunities'] else 'N/A'}
- Main Threat: {swot['threats'][0] if swot['threats'] else 'N/A'}

Provide:
1. KEY INSIGHTS (3-4 bullet points)
2. MARKET IMPLICATIONS
3. STRATEGIC ACTIONS
4. RISK FACTORS

Be specific and actionable."""

        try:
            response = self.call_llm_chat(system_prompt, user_prompt, 1200)
            return response
        except:
            return self.generate_fallback_insights(news_data, topic, sentiment_counts, avg_score)

    def generate_fallback_insights(self, news_data, topic, sentiment_counts, avg_score) -> str:
        """Generate insights without LLM"""

        insights = f"""📊 STRATEGIC INSIGHTS: {topic.upper()}

🔍 KEY FINDINGS:
• Analyzed {len(news_data)} recent news sources
• Overall sentiment: {avg_score:.1f}/100 ({'Positive' if avg_score > 60 else 'Neutral' if avg_score > 40 else 'Negative'})
• Positive coverage: {sentiment_counts.get('Positive', 0)} articles
• Concerns raised: {sentiment_counts.get('Negative', 0)} articles

💼 BUSINESS IMPLICATIONS:
• Market showing {'strong positive momentum' if avg_score > 65 else 'mixed signals' if avg_score > 45 else 'caution indicators'}
• Stakeholder attention is {'high' if len(news_data) > 7 else 'moderate'}
• Industry developments suggest {'growth opportunities' if avg_score > 60 else 'careful monitoring needed'}

🎯 STRATEGIC ACTIONS:
1. Monitor sentiment trends for early warning signals
2. Leverage positive developments for competitive advantage
3. Prepare contingency plans for identified risks
4. Engage stakeholders based on current market perception

⚠️ RISK FACTORS:
• Sentiment volatility: {sentiment_counts.get('Negative', 0)} concerning reports
• Market uncertainty requires adaptive strategies
• Regulatory and competitive landscape changes possible

📈 CONFIDENCE LEVEL: {('High' if len(news_data) > 8 else 'Medium')} based on {len(news_data)} data sources"""

        return insights


In [ ]:
# STEP 6: ENHANCED POLICY ADVISOR
# ============================================================

class AdvancedPolicyAdvisor:
    """Generate executive recommendations"""

    def __init__(self, client):
        self.client = client
        self.model = MODEL_NAME

    def call_llm_chat(self, system_prompt: str, user_prompt: str, max_tokens: int = 1200) -> str:
        """Chat API call"""
        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]

            response = self.client.chat_completion(
                messages=messages,
                model=self.model,
                max_tokens=max_tokens,
                temperature=0.6
            )

            return response.choices[0].message.content
        except Exception as e:
            print(f"Policy Advisor Error: {e}")
            return self.generate_fallback_recommendations()

    def generate_recommendations(self, swot: Dict, topic: str, sentiment_score: float) -> str:
        """Generate strategic recommendations"""

        system_prompt = """You are a senior policy advisor and strategic consultant.
Provide clear, actionable recommendations in a professional consulting memo format."""

        swot_summary = f"""
Strengths: {', '.join(swot.get('strengths', [])[:2])}
Opportunities: {', '.join(swot.get('opportunities', [])[:2])}
Threats: {', '.join(swot.get('threats', [])[:2])}
"""

        user_prompt = f"""Topic: {topic}
Sentiment Score: {sentiment_score:.1f}/100

SWOT Summary:
{swot_summary}

Provide STRATEGIC RECOMMENDATIONS as a consulting memo:

1. EXECUTIVE SUMMARY (2-3 sentences)
2. TOP 3 STRATEGIC PRIORITIES (with rationale)
3. IMPLEMENTATION ROADMAP (short-term and long-term actions)
4. RISK MITIGATION STRATEGIES
5. SUCCESS METRICS & KPIs

Be specific, actionable, and business-focused."""

        try:
            response = self.call_llm_chat(system_prompt, user_prompt)
            return response
        except:
            return self.generate_fallback_recommendations(topic, swot, sentiment_score)

    def generate_fallback_recommendations(self, topic="the analyzed topic", swot=None, sentiment_score=50) -> str:
        """Fallback recommendations"""

        if swot is None:
            swot = {"strengths": [], "opportunities": [], "threats": []}

        return f"""🎯 STRATEGIC RECOMMENDATIONS: {topic.upper()}

📋 EXECUTIVE SUMMARY:
Based on comprehensive analysis of {topic}, the market presents a {'favorable' if sentiment_score > 60 else 'challenging'} landscape.
Strategic action is recommended to capitalize on emerging opportunities while mitigating identified risks.

💼 TOP 3 STRATEGIC PRIORITIES:

1. MARKET POSITIONING OPTIMIZATION
   • Leverage current sentiment trends ({sentiment_score:.0f}/100 score)
   • Build competitive advantages in identified strength areas
   • Timeline: Immediate (0-3 months)

2. RISK MANAGEMENT FRAMEWORK
   • Address potential threats proactively
   • Develop contingency plans for market volatility
   • Timeline: Short-term (3-6 months)

3. INNOVATION & GROWTH STRATEGY
   • Pursue opportunities in emerging market segments
   • Invest in strategic capabilities
   • Timeline: Medium-term (6-12 months)

📅 IMPLEMENTATION ROADMAP:

SHORT-TERM (0-3 months):
• Establish monitoring systems for key indicators
• Initiate stakeholder engagement programs
• Develop detailed action plans for top priorities

LONG-TERM (6-12 months):
• Scale successful initiatives
• Build sustainable competitive advantages
• Expand into identified opportunity areas

⚠️ RISK MITIGATION STRATEGIES:
• Continuous market monitoring and sentiment analysis
• Diversification across multiple strategic initiatives
• Flexible resource allocation for rapid response
• Regular strategy reviews and adjustments

📊 SUCCESS METRICS & KPIs:
• Market sentiment improvement target: +10-15 points
• Strategic initiative completion rate: >80%
• Stakeholder satisfaction scores
• Competitive positioning index
• Risk exposure reduction: 20-30%

🔄 NEXT STEPS:
1. Conduct detailed feasibility analysis
2. Secure stakeholder buy-in
3. Allocate resources and assign ownership
4. Establish tracking and reporting mechanisms
5. Schedule quarterly strategy reviews"""

In [ ]:
# STEP 7: ADVANCED VISUALIZER with Impressive Charts
# ============================================================

class AdvancedVisualizer:
    """Create professional, impressive visualizations"""

    def create_swot_matrix(self, swot_data: Dict) -> go.Figure:
        """Advanced SWOT matrix with detailed view"""

        categories = ['Strengths', 'Weaknesses', 'Opportunities', 'Threats']
        counts = [
            len(swot_data.get('strengths', [])),
            len(swot_data.get('weaknesses', [])),
            len(swot_data.get('opportunities', [])),
            len(swot_data.get('threats', []))
        ]

        colors = ['#27ae60', '#e74c3c', '#3498db', '#f39c12']

        fig = go.Figure()

        # Add bars
        fig.add_trace(go.Bar(
            x=categories,
            y=counts,
            marker=dict(
                color=colors,
                line=dict(color='rgba(255,255,255,0.5)', width=2)
            ),
            text=counts,
            textposition='outside',
            textfont=dict(size=14, color='white'),
            hovertemplate='<b>%{x}</b><br>Factors: %{y}<extra></extra>'
        ))

        fig.update_layout(
            title=dict(
                text='<b>SWOT Analysis Matrix</b>',
                font=dict(size=20, color='white')
            ),
            xaxis=dict(
                title='Strategic Category',
                titlefont=dict(size=14, color='white'),
                tickfont=dict(size=12, color='white'),
                gridcolor='rgba(255,255,255,0.1)'
            ),
            yaxis=dict(
                title='Number of Factors',
                titlefont=dict(size=14, color='white'),
                tickfont=dict(size=12, color='white'),
                gridcolor='rgba(255,255,255,0.1)'
            ),
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='#1e1e1e',
            height=450,
            font=dict(color='white'),
            showlegend=False
        )

        return fig

    def create_sentiment_distribution(self, articles: List[Dict]) -> go.Figure:
        """Sentiment distribution pie chart"""

        if not articles:
            return self.create_empty_chart("No Sentiment Data")

        sentiments = [a['sentiment']['sentiment'] for a in articles]
        sentiment_counts = Counter(sentiments)

        labels = list(sentiment_counts.keys())
        values = list(sentiment_counts.values())
        colors = {
            'Positive': '#27ae60',
            'Negative': '#e74c3c',
            'Neutral': '#95a5a6'
        }
        color_list = [colors.get(label, '#3498db') for label in labels]

        fig = go.Figure(data=[go.Pie(
            labels=labels,
            values=values,
            marker=dict(colors=color_list, line=dict(color='white', width=2)),
            textinfo='label+percent',
            textfont=dict(size=14, color='white'),
            hovertemplate='<b>%{label}</b><br>Articles: %{value}<br>Percentage: %{percent}<extra></extra>'
        )])

        fig.update_layout(
            title=dict(
                text='<b>Sentiment Distribution</b>',
                font=dict(size=20, color='white')
            ),
            paper_bgcolor='#1e1e1e',
            plot_bgcolor='#1e1e1e',
            height=400,
            font=dict(color='white'),
            showlegend=True,
            legend=dict(
                font=dict(size=12, color='white'),
                bgcolor='rgba(0,0,0,0.3)'
            )
        )

        return fig

    def create_sentiment_timeline(self, articles: List[Dict]) -> go.Figure:
        """Sentiment over time (simulated timeline)"""

        if not articles:
            return self.create_empty_chart("No Timeline Data")

        # Create simulated timeline
        scores = [a['sentiment']['score'] for a in articles]
        indices = list(range(len(articles)))

        # Calculate trend line
        z = np.polyfit(indices, scores, 1)
        p = np.poly1d(z)
        trend_line = p(indices)

        fig = go.Figure()

        # Add scatter plot
        fig.add_trace(go.Scatter(
            x=indices,
            y=scores,
            mode='markers+lines',
            name='Sentiment Score',
            marker=dict(
                size=12,
                color=scores,
                colorscale='RdYlGn',
                showscale=True,
                colorbar=dict(title='Score', tickfont=dict(color='white'), titlefont=dict(color='white')),
                line=dict(color='white', width=1)
            ),
            line=dict(color='rgba(255,255,255,0.3)', width=2),
            hovertemplate='<b>Article %{x}</b><br>Score: %{y:.1f}<extra></extra>'
        ))

        # Add trend line
        fig.add_trace(go.Scatter(
            x=indices,
            y=trend_line,
            mode='lines',
            name='Trend',
            line=dict(color='#f39c12', width=3, dash='dash'),
            hovertemplate='Trend: %{y:.1f}<extra></extra>'
        ))

        fig.update_layout(
            title=dict(
                text='<b>Sentiment Trend Analysis</b>',
                font=dict(size=20, color='white')
            ),
            xaxis=dict(
                title='Article Sequence',
                titlefont=dict(size=14, color='white'),
                tickfont=dict(size=12, color='white'),
                gridcolor='rgba(255,255,255,0.1)'
            ),
            yaxis=dict(
                title='Sentiment Score (0-100)',
                titlefont=dict(size=14, color='white'),
                tickfont=dict(size=12, color='white'),
                gridcolor='rgba(255,255,255,0.1)',
                range=[0, 100]
            ),
            plot_bgcolor='#1e1e1e',
            paper_bgcolor='#1e1e1e',
            height=400,
            font=dict(color='white'),
            showlegend=True,
            legend=dict(
                font=dict(size=12, color='white'),
                bgcolor='rgba(0,0,0,0.3)'
            )
        )

        return fig

    def create_sentiment_gauge(self, articles: List[Dict]) -> go.Figure:
        """Advanced sentiment gauge"""

        if not articles:
            avg_score = 50
        else:
            avg_score = np.mean([a['sentiment']['score'] for a in articles])

        fig = go.Figure(go.Indicator(
            mode="gauge+number+delta",
            value=avg_score,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': "<b>Overall Sentiment</b>", 'font': {'size': 20, 'color': 'white'}},
            delta={'reference': 50, 'font': {'size': 14, 'color': 'white'}},
            number={'font': {'size': 40, 'color': 'white'}},
            gauge={
                'axis': {'range': [None, 100], 'tickwidth': 2, 'tickcolor': "white"},
                'bar': {'color': "#3498db", 'thickness': 0.75},
                'bgcolor': "rgba(0,0,0,0.3)",
                'borderwidth': 2,
                'bordercolor': "white",
                'steps': [
                    {'range': [0, 33], 'color': 'rgba(231, 76, 60, 0.3)'},
                    {'range': [33, 66], 'color': 'rgba(243, 156, 18, 0.3)'},
                    {'range': [66, 100], 'color': 'rgba(39, 174, 96, 0.3)'}
                ],
                'threshold': {
                    'line': {'color': "#f39c12", 'width': 4},
                    'thickness': 0.75,
                    'value': avg_score
                }
            }
        ))

        fig.update_layout(
            paper_bgcolor='#1e1e1e',
            font={'color': 'white'},
            height=350
        )

        return fig

    def create_source_distribution(self, articles: List[Dict]) -> go.Figure:
        """Source distribution chart"""

        if not articles:
            return self.create_empty_chart("No Source Data")

        sources = [a['source'] for a in articles]
        source_counts = Counter(sources)

        labels = list(source_counts.keys())
        values = list(source_counts.values())

        fig = go.Figure(data=[go.Bar(
            x=labels,
            y=values,
            marker=dict(
                color='#3498db',
                line=dict(color='white', width=1.5)
            ),
            text=values,
            textposition='outside',
            textfont=dict(size=12, color='white'),
            hovertemplate='<b>%{x}</b><br>Articles: %{y}<extra></extra>'
        )])

        fig.update_layout(
            title=dict(
                text='<b>News Sources Distribution</b>',
                font=dict(size=20, color='white')
            ),
            xaxis=dict(
                title='Source',
                titlefont=dict(size=14, color='white'),
                tickfont=dict(size=10, color='white'),
                tickangle=-45,
                gridcolor='rgba(255,255,255,0.1)'
            ),
            yaxis=dict(
                title='Number of Articles',
                titlefont=dict(size=14, color='white'),
                tickfont=dict(size=12, color='white'),
                gridcolor='rgba(255,255,255,0.1)'
            ),
            plot_bgcolor='#1e1e1e',
            paper_bgcolor='#1e1e1e',
            height=400,
            font=dict(color='white'),
            showlegend=False
        )

        return fig

    def create_empty_chart(self, message: str) -> go.Figure:
        """Empty placeholder chart"""
        fig = go.Figure()
        fig.add_annotation(
            text=message,
            xref="paper", yref="paper",
            x=0.5, y=0.5,
            showarrow=False,
            font=dict(size=16, color='white')
        )
        fig.update_layout(
            paper_bgcolor='#1e1e1e',
            plot_bgcolor='#1e1e1e',
            height=400
        )
        return fig


In [ ]:
# STEP 8: COMPLETE ORCHESTRATOR
# ============================================================

class UltraAIOrchestrator:
    """Master coordinator with all enhanced agents"""

    def __init__(self, hf_token: str):
        self.client = InferenceClient(token=hf_token)
        self.retriever = EnhancedNewsRetriever()
        self.analyst = AdvancedStrategicAnalyst(self.client)
        self.advisor = AdvancedPolicyAdvisor(self.client)
        self.visualizer = AdvancedVisualizer()

    def generate_empty_result(self):
        """Generate empty result when no articles found"""
        return {
            'articles': [],
            'swot': {
                'strengths': ['No data available'],
                'weaknesses': ['Insufficient information'],
                'opportunities': ['Unable to determine'],
                'threats': ['No analysis possible']
            },
            'insights': 'No articles found for this topic. Please try a different search query or check your internet connection.',
            'recommendations': 'Unable to generate recommendations without data. Try different keywords or a broader topic.'
        }

    def run_full_analysis(self, topic: str, num_articles: int = 10):
        """Execute complete multi-agent pipeline"""

        print(f"🔍 Starting analysis for: {topic}")

        # Step 1: Retrieve News
        print("📰 Retrieving news articles...")
        articles = self.retriever.retrieve_news(topic, num_articles)

        if not articles or len(articles) == 0:
            print("⚠️ No articles found")
            return self.generate_empty_result()

        print(f"✅ Retrieved {len(articles)} articles")

        # Step 2: Strategic Analysis
        print("📊 Performing SWOT analysis...")
        swot_analysis = self.analyst.analyze_swot(articles, topic)
        print("✅ SWOT complete")

        # Step 3: Generate Insights
        print("💡 Generating strategic insights...")
        insights = self.analyst.generate_insights(articles, topic, swot_analysis)
        print("✅ Insights generated")

        # Step 4: Policy Recommendations
        print("🎯 Creating policy recommendations...")
        avg_sentiment = sum([a['sentiment']['score'] for a in articles]) / len(articles)
        recommendations = self.advisor.generate_recommendations(
            swot_analysis, topic, avg_sentiment
        )
        print("✅ Recommendations complete")

        print("🎉 Analysis pipeline finished!")

        return {
            'articles': articles,
            'swot': swot_analysis,
            'insights': insights,
            'recommendations': recommendations
        }

In [ ]:
# STEP 9: PREMIUM GRADIO INTERFACE
# ============================================================

def create_premium_interface():
    """Create professional, impressive Gradio interface"""

    # Initialize orchestrator
    orchestrator = UltraAIOrchestrator(HF_TOKEN)

    def analyze_topic(topic, num_articles):
        """Main analysis function with progress"""

        if not topic or topic.strip() == "":
            empty_fig = orchestrator.visualizer.create_empty_chart("Please enter a topic")
            return (
                "<p style='color: #e74c3c;'>⚠️ Please enter a topic to analyze</p>",
                "<p style='color: #e74c3c;'>No SWOT data available</p>",
                "Please enter a topic to begin analysis.",
                empty_fig, empty_fig, empty_fig, empty_fig, empty_fig,
                "No recommendations available."
            )

        try:
            # Run full analysis
            result = orchestrator.run_full_analysis(topic, int(num_articles))

            # Format articles with sentiment badges
            articles_html = """
            <div style='font-family: Arial, sans-serif;'>
                <style>
                    .article-card {
                        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                        padding: 20px;
                        margin-bottom: 20px;
                        border-radius: 12px;
                        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
                        transition: transform 0.3s;
                    }
                    .article-card:hover {
                        transform: translateY(-5px);
                        box-shadow: 0 6px 12px rgba(0,0,0,0.2);
                    }
                    .sentiment-badge {
                        display: inline-block;
                        padding: 5px 12px;
                        border-radius: 20px;
                        font-size: 12px;
                        font-weight: bold;
                        margin-right: 10px;
                    }
                    .positive { background: #27ae60; color: white; }
                    .negative { background: #e74c3c; color: white; }
                    .neutral { background: #95a5a6; color: white; }
                </style>
            """

            for i, article in enumerate(result['articles'], 1):
                sentiment = article['sentiment']['sentiment'].lower()
                sentiment_class = sentiment
                sentiment_score = article['sentiment']['score']

                articles_html += f"""
                <div class='article-card'>
                    <div style='margin-bottom: 10px;'>
                        <span class='sentiment-badge {sentiment_class}'>{sentiment.upper()} ({sentiment_score:.1f})</span>
                        <span style='color: rgba(255,255,255,0.8); font-size: 13px;'>{article['source']} • {article['published']}</span>
                    </div>
                    <h3 style='color: white; margin: 10px 0; font-size: 18px;'>{i}. {article['title']}</h3>
                    <p style='color: rgba(255,255,255,0.9); line-height: 1.6; margin: 10px 0;'>{article['description']}</p>
                    <a href='{article['link']}' target='_blank'
                       style='color: #74ebd5; text-decoration: none; font-weight: bold;'>
                       Read Full Article →
                    </a>
                </div>
                """

            articles_html += "</div>"

            # Format SWOT with modern design
            swot = result['swot']
            swot_html = f"""
            <div style='font-family: Arial, sans-serif;'>
                <style>
                    .swot-grid {{
                        display: grid;
                        grid-template-columns: 1fr 1fr;
                        gap: 20px;
                        margin-top: 20px;
                    }}
                    .swot-card {{
                        padding: 20px;
                        border-radius: 12px;
                        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
                    }}
                    .swot-card h3 {{
                        margin-top: 0;
                        font-size: 20px;
                        margin-bottom: 15px;
                    }}
                    .swot-card ul {{
                        list-style: none;
                        padding: 0;
                        margin: 0;
                    }}
                    .swot-card li {{
                        padding: 10px;
                        margin: 8px 0;
                        background: rgba(255,255,255,0.1);
                        border-radius: 6px;
                        line-height: 1.5;
                    }}
                </style>
                <div class='swot-grid'>
                    <div class='swot-card' style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white;'>
                        <h3>💪 STRENGTHS</h3>
                        <ul>
                            {''.join([f"<li>✓ {s}</li>" for s in swot.get('strengths', ['No data'])])}
                        </ul>
                    </div>
                    <div class='swot-card' style='background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); color: white;'>
                        <h3>⚠️ WEAKNESSES</h3>
                        <ul>
                            {''.join([f"<li>⚡ {w}</li>" for w in swot.get('weaknesses', ['No data'])])}
                        </ul>
                    </div>
                    <div class='swot-card' style='background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); color: white;'>
                        <h3>🎯 OPPORTUNITIES</h3>
                        <ul>
                            {''.join([f"<li>→ {o}</li>" for o in swot.get('opportunities', ['No data'])])}
                        </ul>
                    </div>
                    <div class='swot-card' style='background: linear-gradient(135deg, #fa709a 0%, #fee140 100%); color: white;'>
                        <h3>⚡ THREATS</h3>
                        <ul>
                            {''.join([f"<li>⚠ {t}</li>" for t in swot.get('threats', ['No data'])])}
                        </ul>
                    </div>
                </div>
            </div>
            """

            # Create all visualizations
            swot_chart = orchestrator.visualizer.create_swot_matrix(swot)
            sentiment_dist = orchestrator.visualizer.create_sentiment_distribution(result['articles'])
            sentiment_timeline = orchestrator.visualizer.create_sentiment_timeline(result['articles'])
            sentiment_gauge = orchestrator.visualizer.create_sentiment_gauge(result['articles'])
            source_dist = orchestrator.visualizer.create_source_distribution(result['articles'])

            return (
                articles_html,
                swot_html,
                result['insights'],
                swot_chart,
                sentiment_dist,
                sentiment_timeline,
                sentiment_gauge,
                source_dist,
                result['recommendations']
            )

        except Exception as e:
            error_msg = f"Error during analysis: {str(e)}"
            print(error_msg)
            empty_fig = orchestrator.visualizer.create_empty_chart(error_msg)

            return (
                f"<p style='color: #e74c3c;'>❌ {error_msg}</p>",
                "<p style='color: #e74c3c;'>Analysis failed</p>",
                error_msg,
                empty_fig, empty_fig, empty_fig, empty_fig, empty_fig,
                error_msg
            )

    # Custom CSS for premium look
    custom_css = """
    .gradio-container {
        font-family: 'Inter', sans-serif;
        max-width: 1400px !important;
    }
    .gr-button-primary {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        border: none !important;
        font-size: 16px !important;
        font-weight: 600 !important;
    }
    .gr-button-primary:hover {
        transform: translateY(-2px);
        box-shadow: 0 5px 15px rgba(102, 126, 234, 0.4) !important;
    }
    """

    # Build interface
    with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="AI News & Policy Analyst Pro") as interface:

        gr.Markdown("""
        # 🌍 AI Global News & Policy Analyst Pro
        ### Enterprise-Grade Multi-Agent Intelligence System

        <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white; margin: 20px 0;'>
            <p style='margin: 0; font-size: 16px;'>
                🤖 <b>Powered by Meta Llama 3.1</b> | 📊 <b>Real-Time SWOT Analysis</b> |
                💡 <b>Strategic Recommendations</b> | 📈 <b>Advanced Visualizations</b>
            </p>
        </div>
        """)

        with gr.Row():
            with gr.Column(scale=3):
                topic_input = gr.Textbox(
                    label="🎯 Analysis Topic",
                    placeholder="Examples: AI regulation fintech India, semiconductor export controls impact, renewable energy policy 2025...",
                    lines=2,
                    info="Enter any topic for comprehensive strategic analysis"
                )
            with gr.Column(scale=1):
                num_articles = gr.Slider(
                    minimum=5,
                    maximum=15,
                    value=10,
                    step=1,
                    label="📚 Articles to Analyze",
                    info="More articles = deeper analysis"
                )

        analyze_btn = gr.Button("🚀 Run Complete Analysis", variant="primary", size="lg")

        gr.Markdown("---")

        with gr.Tabs():
            with gr.Tab("📰 News Intelligence"):
                gr.Markdown("### Latest News with Sentiment Analysis")
                news_output = gr.HTML()

            with gr.Tab("📊 SWOT Matrix"):
                gr.Markdown("### Strategic SWOT Analysis")
                swot_output = gr.HTML()
                with gr.Row():
                    swot_chart = gr.Plot(label="SWOT Visualization")

            with gr.Tab("💡 Strategic Insights"):
                gr.Markdown("### Executive Insights & Analysis")
                insights_output = gr.Textbox(
                    label="Detailed Strategic Insights",
                    lines=20,
                    show_copy_button=True,
                    info="AI-generated strategic analysis"
                )

            with gr.Tab("📈 Analytics Dashboard"):
                gr.Markdown("### Advanced Data Visualizations")
                with gr.Row():
                    sentiment_dist_chart = gr.Plot(label="Sentiment Distribution")
                    sentiment_gauge_chart = gr.Plot(label="Overall Sentiment Score")
                with gr.Row():
                    sentiment_timeline_chart = gr.Plot(label="Sentiment Trend Analysis")
                    source_dist_chart = gr.Plot(label="News Sources Distribution")

            with gr.Tab("🎯 Policy Recommendations"):
                gr.Markdown("### Strategic Recommendations & Action Plan")
                recommendations_output = gr.Textbox(
                    label="Executive Recommendations",
                    lines=20,
                    show_copy_button=True,
                    info="Consulting-style strategic recommendations"
                )

        # Connect interactions
        analyze_btn.click(
            fn=analyze_topic,
            inputs=[topic_input, num_articles],
            outputs=[
                news_output,
                swot_output,
                insights_output,
                swot_chart,
                sentiment_dist_chart,
                sentiment_timeline_chart,
                sentiment_gauge_chart,
                source_dist_chart,
                recommendations_output
            ]
        )

        # Examples section
        gr.Markdown("""
        ---
        ### 💼 Example Analysis Topics

        <div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 15px; margin-top: 20px;'>
            <div style='padding: 15px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 8px; color: white;'>
                <b>🏦 Fintech</b><br/>
                "AI regulation impact on Indian fintech startups"
            </div>
            <div style='padding: 15px; background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); border-radius: 8px; color: white;'>
                <b>💻 Technology</b><br/>
                "Semiconductor export controls tech industry"
            </div>
            <div style='padding: 15px; background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); border-radius: 8px; color: white;'>
                <b>🌱 Climate</b><br/>
                "Renewable energy policy developments 2025"
            </div>
            <div style='padding: 15px; background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%); border-radius: 8px; color: white;'>
                <b>🏥 Healthcare</b><br/>
                "Healthcare AI implementation regulations"
            </div>
        </div>

        ### 🏗️ System Architecture

        **5-Agent Pipeline:**
        1. 🔍 **Enhanced News Retriever** → Multi-source aggregation (Google News, Bing News) + Sentiment Analysis
        2. 📊 **Strategic Analyst** → SWOT Analysis + Business Framework Application
        3. 💡 **Insight Generator** → Pattern Recognition + Market Intelligence
        4. 🎯 **Policy Advisor** → McKinsey-style Recommendations + Implementation Roadmap
        5. 📈 **Advanced Visualizer** → Interactive Dashboards + Executive Reports

        **Technology Stack:**
        - 🤖 LLM: Meta Llama 3.1-8B-Instruct (HuggingFace)
        - 📡 Data: RSS Feeds, News APIs, Sentiment Analysis
        - 📊 Viz: Plotly (Interactive Charts)
        - 🎨 UI: Gradio (Enterprise Theme)

        ---

        <div style='text-align: center; padding: 20px; background: #f8f9fa; border-radius: 10px;'>
            <p style='margin: 0; color: #2c3e50;'>
                <b>Built for Hackathon Excellence</b> | Real-time Intelligence | Production-Ready Architecture
            </p>
        </div>
        """)

    return interface


In [ ]:
# STEP 10: LAUNCH THE APPLICATION
# ============================================================

if __name__ == "__main__":
    print("=" * 60)
    print("🚀 AI NEWS & POLICY ANALYST PRO - INITIALIZING")
    print("=" * 60)
    print("📡 Connecting to Hugging Face API...")
    print(f"🤖 Model: {MODEL_NAME}")
    print("🔧 Loading all agents...")
    print("=" * 60)

    # Create and launch
    interface = create_premium_interface()

    print("\n✅ ALL SYSTEMS OPERATIONAL")
    print("🌐 Launching interface...")
    print("=" * 60)

    interface.launch(
        share=True,          # Creates public shareable link
        debug=True,          # Enable debugging
        server_port=7860,    # Default Gradio port
        show_error=True      # Show detailed errors
    )

    print("\n🎉 APPLICATION RUNNING!")
    print("📱 Access via the URL above")
    print("🔗 Share link is publicly accessible")
    print("=" * 60)

🚀 AI NEWS & POLICY ANALYST PRO - INITIALIZING
📡 Connecting to Hugging Face API...
🤖 Model: meta-llama/Meta-Llama-3-8B-Instruct
🔧 Loading all agents...

✅ ALL SYSTEMS OPERATIONAL
🌐 Launching interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4f3a1d5fed1f575088.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
